In [ ]:
import matplotlib.patches as mpatches
import numpy as np
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
params = {
   'axes.labelsize': 12,
   'font.size': 10,
   'legend.fontsize': 11,
   'xtick.labelsize': 10,
   'ytick.labelsize': 12,
   'text.usetex': False,
   'figure.figsize': [10, 2] # instead of 4.5, 4.5
   }
plt.rcParams.update(params)

## helper function

In [ ]:
def autolabel(rects, ses):
    # attach some text labels
    for rect, se in zip(rects, ses):
        height = rect.get_height()
        plt.axes().text(rect.get_x()+rect.get_width()/2., height+1.03*se, '{:2.3f}'.format(float(height)).lstrip('0'),
                ha='center', va='bottom',fontsize=10)

## Fixed Parameter

In [ ]:
# experiment
exp_all = ['mysseg_all_old','mysseg_all']
exp_label = 'Effect of Adding Dataset Schema'
model = 'multi_srm'

# dictionaries
cl_dict = {0:1/225,1:2/297,2:2/297,3:2/1973} # chance level of each dataset
feat_dict = {'multi_srm':[75,75,100],'all_srm':[75,75,50],'indv_srm':[75,75,50],'all_ica':[50,25,25],'indv_ica':[50,25,25],\
'all_gica':[100,50,25],'indv_gica':[100,50,25],'all_dict':[25,25,25],'indv_dict':[25,25,25]}
roi_dict = {'dmn':0,'pt':1,'eac':2}
# change multi_srm name!
md_dict = {'avg':'MNI','multi_srm':'MDSRM','ica':'ICA','gica':'Group-ICA','dict':'Dict','srm':'SRM'}
ds_dict = {0:'GreenEyes',1:'Milky',2:'Vodka',3:'Sherlock'}
exp_dict = {'mysseg':'Half Training Datasets','mysseg_all':'All Training Datasets'}

# paths 
output_path = '../../output/figures/{}/' #exp
if not os.path.exists(output_path.format(exp_all[1])):
    os.makedirs(output_path.format(exp_all[1]))
output_file = output_path+'{}_ds{}' #roi,ds

## Plotting Parameter

In [ ]:
roi_all = ['dmn','pt','eac']
max_accu = 0.9
total_ds = list(range(12))
datasets = [0,1,2,3]

#exp,(total_ds),roi,model,ds
input_file = ['../../output/accu_bar/{}/'+str(total_ds)+'/{}/{}_ds{}.npz','../../output/accu_bar/{}/{}/{}_ds{}.npz']

## Aggregate Accuracy

In [ ]:
all_mean = [] # length # of roi
all_se = [] # length # of roi
# aggregate each roi
for roi in roi_all:
    roi_mean = []
    roi_se = []
    for ds in datasets:
        for infile,exp in zip(input_file,exp_all):
            ws=np.load(infile.format(exp,roi,model,ds).replace(' ',''))
            roi_mean.append(ws['mean'].item())
            roi_se.append(ws['se'].item())
    all_mean.append(roi_mean)
    all_se.append(roi_se)

## Plot all roi

In [ ]:
width=1
nmodel = len(exp_all)
ndata = len(datasets)
group_width = 1.25*width*nmodel
center_all = np.linspace(0,group_width*(ndata-1),ndata)
# set colors
color_all = ['dodgerblue','red']

for m,roi in enumerate(roi_all):
    ds_idx = []
    ds_name = []
    plt.figure()
    # configure bars
    for i,(ds, center) in enumerate(zip(datasets,center_all)):
        mean = list(all_mean[m][i*nmodel:(i+1)*nmodel])
        se   = list(all_se[m][i*nmodel:(i+1)*nmodel])    
        idx = np.arange(center,center+nmodel*width,width)
        error_config = {'ecolor': '0','capsize':3}   
        rects = plt.bar(idx, mean, yerr=se, align='center', error_kw=error_config, width = width-0.1)
        # set colors
        for rect_i in range(len(rects)):
            rects[rect_i].set_color(color_all[rect_i])
        autolabel(rects, se)
        ds_idx.append(center+(nmodel-1)*width/2)        
        ds_name.append(ds_dict[ds])                     
    plt.xticks(ds_idx,ds_name)
    
    plt.ylabel('Accuracy')
    left_lim = center_all[0]-0.5*width-0.5
    right_lim = center_all[-1]+(nmodel-0.5)*width+0.5
    plt.xlim([left_lim,right_lim])
    plt.ylim([0,max_accu])

    # plot chance accuracy of each dataset
    for d,center in enumerate(center_all):
        cl = cl_dict[datasets[d]]
        line = plt.plot([center-width, center+(nmodel+1)*width], [cl, cl], 'k-.', linewidth=2)
       
    # Add texts
    plt.text((right_lim+left_lim)/2, 1.01*max_accu, exp_label+', ROI: '+roi.upper(),fontsize=12,horizontalalignment='center', verticalalignment='bottom')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text((right_lim+left_lim)/2, 0.93*max_accu, 'k = '+str(feat_dict[model][roi_dict[roi]]),bbox=props,fontsize=12,horizontalalignment='center', verticalalignment='top')
    
    # legend
    legend_handle = []
    for patch_i in range(len(exp_all)):
        legend_handle.append(mpatches.Patch(color=color_all[patch_i], label=exp_dict[exp_all[patch_i]]))    
    l1 = plt.legend(handles=legend_handle,loc=2,ncol=1)
    ax = plt.gca().add_artist(l1)
    plt.legend(line ,['chance'],loc=1,ncol=1)

            
#     plt.savefig(output_file.format(exp,roi,datasets).replace(' ','')+'.eps', format='eps', dpi=200,bbox_inches='tight')
    plt.savefig(output_file.format(exp,roi,datasets).replace(' ','')+'.pdf', format='pdf', dpi=200,bbox_inches='tight')